In [1]:
!pip3 install --upgrade wandb transformers --quiet

from transformers import TFRobertaForSequenceClassification, TFBertForSequenceClassification, TFXLNetForSequenceClassification, TFXLMRobertaForSequenceClassification, TFDistilBertForSequenceClassification, TFAlbertForSequenceClassification
import tensorflow as tf
import numpy as np
import wandb
import os
from sklearn.calibration import calibration_curve
from sklearn.metrics import f1_score, accuracy_score, roc_curve, auc, precision_recall_curve, confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_type = "xlnet"
model_name = "xlnet-large-cased"
project_name = "XLNet-TPU"
lr = 2e-6
batch_size = 8
epochs = 20

npy = f'/content/drive/MyDrive/CJPR/NPY/{model_type}/{model_name}'

In [4]:
folder = f"{npy}/plots_L{lr}_E{epochs}_B{batch_size}"

if not os.path.exists(folder):
        os.makedirs(folder)

In [5]:
key="d75571bf9259088cd0a735d5f9e10de08e105a99"
wandb.login(key=key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
wandb.init(
            project=project_name,
            name=f"{model_name}_L{lr}_B{batch_size}_E{epochs}",

            config={
            "architecture": "TPU",
            "dataset": "ILDC",
            "learning_rate": lr,
            "epochs": epochs,
            "optimizer": "adam",
            "loss": "sparse_categorical_crossentropy",
            "batch_size": batch_size,
            "model_name": model_name
            }
        )

wandb: Currently logged in as: chagantivenkataramireddy4 (ram777). Use `wandb login --relogin` to force relogin


In [7]:
train_input_ids = np.load(f'{npy}/train_input_ids.npy')
train_attention_masks = np.load(f'{npy}/train_attention_masks.npy')
train_labels = np.load(f'{npy}/train_labels.npy')

validation_input_ids = np.load(f'{npy}/validation_input_ids.npy')
validation_attention_masks = np.load(f'{npy}/validation_attention_masks.npy')
validation_labels = np.load(f'{npy}/validation_labels.npy')

test_input_ids = np.load(f'{npy}/test_input_ids.npy')
test_attention_masks = np.load(f'{npy}/test_attention_masks.npy')
test_labels = np.load(f'{npy}/test_labels.npy')

In [8]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
with tf.device('/job:worker'):
    with strategy.scope():
        model = TFRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
        print('model loaded')
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
        model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
        print('starting fitting')

        history_callback = model.fit(
            [train_input_ids, train_attention_masks], train_labels,
            epochs=epochs, batch_size=batch_size,
            validation_data=(
                [validation_input_ids, validation_attention_masks], validation_labels
            ),
            verbose=1
        )

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

You are using a model of type xlnet to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


tf_model.h5:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

In [ ]:
val_loss = history_callback.history['loss']
val_accuracy = history_callback.history['accuracy']

In [ ]:
print(f"Validation Accuracy: {val_accuracy}\nValidation Loss: {val_loss}")

In [ ]:
epochs = list(range(1, epochs+1))
print(epochs)

In [ ]:
plt.figure()
plt.plot(epochs, val_loss, 'b-', label='Validation Loss')
plt.plot(epochs, val_loss, 'ro')
plt.title('Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(f"{folder}/validation_loss_curve.png")

plt.show()

In [ ]:
plt.figure()
plt.plot(epochs, val_accuracy, 'r-', label='Validation Accuracy')
plt.plot(epochs, val_accuracy, 'bo')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig(f"{folder}/validation_accuracy_curve.png")

plt.show()

In [ ]:
plt.figure()
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, val_accuracy, 'b-' , label='Validation Accuracy')
plt.title('Validation Accuracy vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig(f"{folder}/validation_accuracy_vs_loss.png")

plt.show()

In [ ]:
predicted = model.predict([test_input_ids, test_attention_masks])

In [ ]:
predicted_labels = [np.argmax(i) for i in predicted[0]]

In [ ]:
predicted_probabilities = tf.nn.softmax(predicted[0], axis=-1).numpy()

In [ ]:
predicted_probabilities_class_1 = predicted_probabilities[:, 1]

prob_true, prob_pred = calibration_curve(test_labels, predicted_probabilities_class_1, n_bins=10, strategy="uniform")

In [ ]:

fig, ax = plt.subplots()
ax.plot(prob_pred, prob_true, marker='o', linestyle='--', color='red', label='Model')
ax.plot([0, 1], [0, 1], linestyle='--', color='blue', label='Perfectly calibrated')
ax.set_xlabel('Mean Predicted Probability')
ax.set_ylabel('Fraction of Positives')
ax.set_title('Calibration Curve')
ax.legend()
ax.grid(True)
plt.savefig(f"{folder}/calibration_curve.png")

In [ ]:
wandb.log({"calibration_curve": wandb.Image(fig)})

In [ ]:
f1 = f1_score(test_labels, predicted_labels, average='macro')
print(f1)
acc = accuracy_score(test_labels, predicted_labels)
print(acc)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc

fpr, tpr, _ = roc_curve(test_labels, predicted_probabilities[:, 1])

roc_auc = auc(fpr, tpr)

In [ ]:
mean_fpr = np.mean(fpr)
print(mean_fpr)

In [ ]:
mean_tpr = np.mean(tpr)
print(mean_tpr)

In [ ]:


# Plot ROC Curve
fig, ax = plt.subplots()
ax.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve')
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve')
ax.legend(loc="lower right")
plt.savefig(f"{folder}/roc_curve.png")

In [ ]:
wandb.log({"roc_curve": wandb.Image(fig)})

In [ ]:
print("AUC: {:.2f}".format(roc_auc))

In [ ]:
from sklearn.metrics import precision_recall_curve,auc as calculate_auc

precision, recall, _ = precision_recall_curve(test_labels, predicted_probabilities[:, 1])
pr_auc = calculate_auc(recall, precision)

fig, ax = plt.subplots()
ax.plot(recall, precision, color='darkorange', lw=2, label='Precision-Recall curve')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall Curve')
ax.legend(loc="lower left")
plt.savefig(f"{folder}/precision_recall_curve.png")

In [ ]:
wandb.log({"precision_recall_curve": wandb.Image(fig)})

In [ ]:
print(pr_auc)

In [ ]:

confusion = confusion_matrix(test_labels, predicted_labels)
class_names = ["Rejected", "Accepted"]

fig, ax = plt.subplots()
sns.heatmap(confusion, annot=True, fmt='d', cmap="Blues", xticklabels=class_names, yticklabels=class_names)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(f"{folder}/confusion_matrix.png")

In [ ]:
wandb.log({"confusion_matrix": wandb.Image(fig)})

In [ ]:
import json

result_data = {
    "F1 Score": f1,
    "Accuracy": acc,
    "AUC": roc_auc,
    "PR AUC": pr_auc,
    "Confusion Matrix": confusion.tolist(),
    "Mean FPR": mean_fpr,
    "Mean TPR": mean_tpr,
    "Validation Accuracy": val_accuracy,
    "Validation Loss": val_loss
}

json_filename = f"{folder}/metrics.json"

with open(json_filename, "w") as json_file:
    json.dump(result_data, json_file, indent=4)

In [ ]:
wandb.finish()

In [ ]:
# save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
# model.save(f'{folder}/model', options=save_locally)
# with strategy.scope():
#     load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
#     model = tf.keras.models.load_model(f'{folder}/model', options=load_locally)

# model.save(f'{folder}/model.h5')